In [1]:
from ctm_python_client.core.workflow import *
from ctm_python_client.core.comm import *
from ctm_python_client.core.credential import *
from aapi import *
from ctm_python_client.ext.viz import get_graph
#from IPython.display import display, JSON
#import getpass


import os
BASE_PATH = os.path.abspath("")
URL='https://se-dev-aapi.sandbox.us1.controlm.com/automation-api'
#with open(BASE_PATH + "/secrets", "r") as fp:
#    ctm_uri = fp.readline().strip()
##    ctm_user = fp.readline().strip()
#    ctm_pwd = fp.readline().strip()

#my_environment = Environment(ctm_uri, username=ctm_user, password=ctm_pwd, mode=EnvironmentMode.ONPREM)
my_environment = Environment.create_saas(URL,
                                         api_key='UFJEU0xDOmUzMmRiZTFkLTU0MDEtNGZmOS1hNTUyLTk3NjQzMGFhYmM4NjpKS3pqVWl4RDZrU3RXY1ZoSFZPelFES2t6YnpIS0V2SFZxVkVUN21VaWR3PQ==')

# Creation du Workspace
workflow = Workflow(my_environment,
                    WorkflowDefaults(application='MPE',
                                     sub_application='demo4')
                    )

Myfolder = "mpe-demo4-folder1"
MyCTM = "IN01"
MySITE = "mpe_site1"
# Folder
folder = Folder(Myfolder,
                site_standard=MySITE,
                controlm_server=MyCTM,
                description='NEOM from Python demo4',
                order_method=Folder.OrderMethod.Manual,
                variables=[{'FICHIER_SQL': '/tmp/SQL.txt'},{'MYFILE': 'demo123.txt'}])
workflow.add(folder)


#  Jobs
myJob1 = workflow.add(
    JobCommand(Myfolder + "-linux1", command='df -h', run_as='bmc',
               host='zzz-linux-agents'),
    inpath=Myfolder
)


myJob2 = workflow.add(
    JobDatabaseEmbeddedQuery(Myfolder + "-SQL_PG", query='SELECT table_name FROM information_schema.tables',
                             connection_profile='MPE_PGSQL', host='mpe-linux',
                             output_execution_log='N', output_sql_output='Y', sql_output_format='CSV', run_as='MPE_PGSQL',
                             output=Job.Output(
                                 operation=Job.Output.Operation.Copy, destination='Fichier_SQL')
                             ),
    inpath=Myfolder,
)
action = ActionOutput(Myfolder + "-SQL_PG",
                      operation=ActionOutput.Operation.Copy, destination='%%FICHIER_SQL')
myJob3 = workflow.add(
    JobCommand(Myfolder + "-linux2", command='sleep 10', run_as='bmc',
               host='zzz-linux-agents',),
    inpath=Myfolder
)
myJob4 = workflow.add(
    JobFileTransfer(Myfolder + "-MFT-S3",
                    connection_profile_src='MPE_MFT_cent', 
                    connection_profile_dest='ZZZ_MFT_S3',
                    s3_bucket_name='zzz-file-upload-s3',
                    number_of_retries='5', host='mpe-linux',
                    file_transfers=[
                        FileTransfer(src='/home/ctmag/demo123.txt', dest='/incoming/',
                                     transfer_type=FileTransfer.TransferType.Binary,
                                     transfer_option=FileTransfer.TransferOption.SrcToDest),
                    ]
                    ),
    inpath=Myfolder
)

myJob5 = workflow.add( 
    JobAWSLambda(Myfolder + "-AWS-lambda",
                   connection_profile='ZZZ_AWS',
                   function_name='mpe-test1',
                   version='$LATEST',
                   host='zzz-linux-agents',
                   payload='{"Number1": 974,"Number2": 78,"filedemo":"%%MYFILE"}',
                   append_log=True
                   ),
    inpath=Myfolder             
)

myJobadf = JobAzureDataFactory()  
JobQDF = workflow.add(
    myJobadf,
    inpath=Myfolder             
)



jobSLA = workflow.add(
    JobSLAManagement(Myfolder + "-SLA",
                     service_name='mpe-demo-python',
                     run_as='ctmagent',
                     service_priority='1',
                     host='zzz-linux-agents',
                     job_runs_deviations_tolerance='1',
                     complete_in=JobSLAManagement.CompleteIn(time='00:02'),
                     average_run_time_tolerance=JobSLAManagement.AverageRunTimeTolerance(
                         average_run_time='10',
                         units=JobSLAManagement.AverageRunTimeTolerance.Units.Minutes
                     )),
    inpath=Myfolder
)


# Dependances
workflow.connect(myJob1, myJob2)
workflow.connect(myJob1, myJob3)
workflow.connect(myJob2, myJob4)
workflow.connect(myJob3, myJob4)
workflow.connect(myJob4, myJob5)
workflow.connect(myJob5, jobSLA)


if workflow.build().is_ok():
    print('The workflow is valid!')

else:
    print('The workflow is NOT valid!')
    print(workflow.build().errors)
#workflow.dumps_json()
#display(workflow.dumps_json())
#workflow.dumps_json(indent=4)
get_graph(workflow)
#graph = get_graph(workflow)
#graph.render("test_graph", format='png')



TypeError: JobAzureDataFactory.__init__() missing 1 required positional argument: 'object_name'

## Deploy

In [ ]:
if workflow.deploy().is_ok():
    print('The workflow was deployed to Control-M!')
else:
    print('The workflow is NOT deployed!')
    print(workflow.deploy().errors)


## Run

In [ ]:
print('Run')
run = workflow.run()
run.print_statuses()

In [ ]:
run.print_statuses()
#run.set_to_ok('mpe-AMSA-demo4/mpe-AMSA-demo4-linux1')
#run.set_to_ok('mpe-AMSA-demo4/mpe-AMSA-demo4-SQL_PG')
#run.set_to_ok('mpe-AMSA-demo4/mpe-AMSA-demo4-MFT-S3')
#run.print_output('mpe-demo4-folder1/mpe-demo4-folder1-SQL_PG')
#run.print_output('mpe-AMSA-demo4/mpe-AMSA-demo4-AWS-lambda')
#run.set_to_ok('mpe-AMSA-demo4/mpe-AMSA-demo4-SQL_2_LINUX')
